In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.acdc.file_names import project, Opts
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Opts()
# may not need this if pick fm selection
picks = [{'subj': 'sub-acdc117', 'session': 'ses-1', 'run': '1',},  
        ]
input_selkey = widgets.Dropdown(
    options=get_h5_keys(opts.info_fname, key='auto_qc'),
    #value='2',
    description='Subject DWI to qc:',
    disabled=False,
)
input_selkey.layout.width = '600px'
input_selkey

/home/toddr/Software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


Dropdown(description=u'Subject DWI to qc:', layout=Layout(width=u'600px'), options=('/sub-acdc117/ses-1/dwi/auto_qc',), value='/sub-acdc117/ses-1/dwi/auto_qc')

In [2]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
    with open('fslview_cmd.sh', 'w') as f:
        f.write('fslview ' + fslv_names + ' &\n')
all_dwi_auto_qc['itopup_isnum'] = all_dwi_auto_qc['itopup'] == np.number
# limit to only active rows with mask
all_dwi_auto_qc['itopup_isnum'] = all_dwi_auto_qc['itopup'].str.isnumeric()
all_dwi_auto_qc['itopdn_isnum'] = all_dwi_auto_qc['itopdn'].str.isnumeric()
all_dwi_auto_qc.loc[all_dwi_auto_qc.itopup_isnum, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc.itopdn_isnum, 'itopdn_visqc'] = True
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)

In [3]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600)
        ]
box_layout = widgets.Layout(display='flex', flex_flow='row', align_items='stretch', border='solid', width='100%')
dwi_box = widgets.Box(children=items+[dwi_w], layout=box_layout)
dwi_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xbf\xbf\xbf\x1f\x1f\x1f___???\xdf\xdf\xdf\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xf5Uk\xc9\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\x9d\t\x96\xaa\xba\x16@?]&\xc7\x00n5\xaf\xacn\xc1\xfc\x87\xf0M\xdf\x10T,D\xa2{\xaf\xf7n)\x02\' \xd9&!\t\xff\x1b\x01\x00\n\xe1\x7f\xf7N\x00\x00\xc0\xa5 ,\x00(\x06\x84\x05\x00\xc5\x80\xb0\x00\xa0\x18\x10\x16\x00\x14\x03\xc2\x02\x80b@X\x00P\x0c\x08\x0b\x00\x8a\xe1\xa9\x845\x0c\xcdJ{j\x86a\xa5=\xdd\x94\xc1\xd24m\xad\x17\xf5CJ;\xce\x1eO\xd56\x8d\xdc\xb6J\x96\xd7}w\\\xdeM\x96g\xb8\xef\x99j\xfc\x19HS[\r\xc9\xe5\x909YKc\x9dYC\xb4\xcd\xc5W\xe0z\xd7\xeac\x81\xb0\xae\xe2\xe4\xc5)\xa2\x0f\xed\xbbD\x13+%$\x1f\xd3\x12\x8bI\xc8E\x0b\x84\xd5\x07\xd9\xbd\x0f\x96\xd7\x9d_~VY\xb7\x13Vp\xc8U{\x14h\xce\x9fMt\xa4\x9d\x08>R\x07\x11\x8a)s\xb2\x16q\xf6H\xc5\x92/>{\xadV\xfawB\xf8%\xb5t`\x17\x1e\xf8%K\n\x06a]\xc5\xc9\x8b\xb3\x8a>\xac6\x11V5\'\xacV\xa1rg#\xe4\x8azA\xeb>R\xd9<s<\xa23\x05\x93\xae\x893{\x15-\xef\xd3\xed\x12n\',w\xc8\xc2\t\xd4\'\xd2G7G\xa9R\x1b|\xfd\xc2\xc9\xda\x929Y\x8b8{\xa4\xc7s\xde\xf5\x97z#s\xad\xd6N\xbf\xee\xa4w\x93\x03\xbfdI\xc9 \xac\xab8uq\x8a\xe8C\xf7\xae\x0bX\xbd\xbc/\xf2\t\n\xe2T\xc75\xba\xe0\xa3x\x83\xe9\xe6r\x8fC/\xf4\xeb>\xc8\xc1\xb5\xbc\xf6u\xc9D\x15\xb5\xce\xd4\x9en&,\x7f\xc82\x15]\xdf\xb7\x19\xcd\x84\xd1\xa5h\xab\xf0]\x1b\x7f\r\'N\xd6E\x9c=\xd2\xe3\n\xe2\xf4\x1a\xf9\xd4\x18je\xd8\xbem\xbch;\xbd$P\xf1%K\x8a\x06a]\xc5\x89\x8bS^\xec\xc3\xcc;\xbb\xf0l6_H6\xca\x18\x1f\xb0\xcc\xe0Aq\xe8\x9c\xb0\xba(\xf7\x0b\x9f\x83\xa3\xdd\xb4g3\xf6\xad\x84\xe5\x0f\xf9x6\x1bu:eI+)\xf0E\xd1\xab\xb0Du\\W\xfa6\xf8\x1eN\x9c\xac\x8b\xb8DX\x97\xefmz\xadv\xb6\x90T\xd9\x9f\x89\xca|G\xfe\xc0/YR6\x08\xeb*\xe6\xae\xbd\xb6\t+|\xf1;\x87X\xf7\xd2\x99\x89"\x89\x0e\xb8\x8e\xde\x9d\x11V\x95\x94Vd\x0eV\xc5\x93~R\x8f\nW\x9br\x13aE\x87\xdc:\xed\x88\xc9\x17\x1c\x17v\x03\xbb\xaaU#\x83\x9d:Y\x17qca\x05)\xb2\xdfAc\x8f\\\xd8sq\xc9\x92\xb2y,a\x99FIs\xfd\x06\xdf\xb9~)\xff\x15\xb2)\xb93u\x9d\x18\xd1G\x9b\xfb\x1dV\xc1\x1a\xb2\x01Z\x04\xd7^\xbc\x86me\xc8\xbdst\x99RI\x95\x86N"g\xd6\xb0\xccDI\xce\xc0\xa8\x0b\x15\xe1V\xf1>&;\x8d#\xd9\xfc\x92\xd4k\xda\xb0\x9a\x95K\xa8\xda\xb3^\x94\x1e\\\xefv\xa1?\xa9\x9c\xfdz\xa3\xf4\xb9c\x8e\x0e\xb9\xf3i\xcf\x1d\x86\x7f\x13\nKG\x88\xce\xda\x89\x93\xe5S\x14\x1est\x14A\xac|\xaa\x870\xcd\xf1\xa6\xfa\x1c5\xd1\x99\x9c\\\xabA\n\x8c\x98ET\xef\xab.\\\x12a"\x8c\xb3\x17\xf4\xeex$a\xf9FI\xfd\xbb\x99\x13\x96]\xa7\x99f|\x7f\xff\xcc\xfe\xec\n\xbbC\xb3v\xed\xde\xda\xefw\xb2Fu\xc4}\xf9\xf1;K5-\x94\xa4)\x0f\xf6+f\xd6\x086\xceFI\xcf\x80\x0e\xecKv\xa7\x85UO\xabz:\x07\xd7I\x02D]\xc7\x07\x93&T\xee\xb9\x9b;8[\xb1\xd1\xc9\x92\xed,U\x18k\xf6\x98\xa3C\xee:\xf7\xf1\xa4\xd7@t\\}\x90c\xb5w#\xdd\x9e8Yq\xe1M[\'\xfd\xeam\xac\xb9T\x07\xc2J7\x95[\xf6\xc3\x90\n+\xbeV\xdb\xc0\xa0:T\x90D\xf3\xf2\x92%!.\x82\x98\xbb\xa0w\xc7\x03\tK\x9e\xeb\xa6\xede!H\x7f39a\xa9\xdf>\x95\x8

In [4]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xdf\xdf\xdf___???\xbf\xbf\xbf\x1f\x1f\x1f\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xa8*\xe1\x90\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\xdd\t\x96\xa2\xda\xb6\x05\xd0Ou:G3\xde\xc0\xfe7\xe1K]\x1b\x92i\x10\xb9\xe3\xce9\xde\xbb\x19\xea\x91\r\x08\xcbC\xe9\xff=\x00\x82\xf8\xbf\x9f\x1e\x01\x80w\t, \x0c\x81\x05\x84!\xb0\x800\x04\x16\x10\x86\xc0\x02\xc2\x10X@\x18\x02\x0b\x08\xe3w\x07V\xd3\xa4\x0f\r)5\xcd\x87\x86\xf4\xad\x9aQJE\xd5?U7[\xc5\xe3tz\xb2"\xa5\xf6\xbd\xd9\xe6\xf9\xaa.\x9f\xcf\x97\xbb\xe7\x0f\xdc<\xa7\xd2<\xc9\x9b\xd1\xcb\x9a\xcd\xc7\x7f0s\xae\x96\xfa\xa2E^\xa4u\xc9\x9f\xf5\xb9\xc5\xff\xdf!\xb0\xde\xf3ra\xcdW/\x8e\x8f61\xf1\xa1\x119\xae9Z\x07S\xde>u!\xb0\xea\xc5\xda_/\x9e\xaf\xca\xf9\xf9/#\xeb\x9b\x02\xabj\xb3\xa0\xdcWO\xabI+\xf3\xf9\x95n\xa4\x97\xc1t0s.\xf9r\xca\xf2+\x1ft]\x14\x97\xc7\xe0+UQ\xb6\xb9=N\xf4\xd1\xe2\x7f6\x1f\xa3\x10X\xefy\xb9\xb0f\xab\x17\xb3[\x02+;\x0b\xac\xa2\xd3\xad\xad\xa9])\xb3\xfe\x89bz\xa9\xeb\x87\x1cLO^\x0e\xfd\x942\xad\xd7\xfdl\xf5|\xbd}\xdf\xc6\xf7\x04\xd6\x18\x99\xe56g\xd24Y\xdd\xe8\xa5\xe9\x85|\n\xe7\xd1\xc1\xcc\xb9\xe4\xcb){\xce\xe3\xb2~7\x0b>?\x9f\xf2\xe9{e\x98K\x07\x8b\xff\xe9|\x8cB`\xbd\xe7\xd5\xe2\x95\xaf^\x9c\x1e\x95\x0b\x1f\xef\x9c\xe7\xc7#\xb4\xa8\x93=[\x94\x8b\x97\xd6o\xd8\xbf\xbd\x1dbS\xf7\xcbq^/\xd6\xe8\xaa]\xbe\xfb/\xed\xae\xab\xf5\xc5\xd6\xd4\xb7\x04V[\xb7\xa8\x8bf?\x1f\x97\xe5\xdad\xcd\x16\x0f\x8au\xf3\x173\xe7-_N\xd9\xb3\xc1\xfb9\xf0\xf1\xf9\xd4}\x80\xcf\x99\xd4\xce\xab~\xda\xf6s\xeb|>F!\xb0\xde\xf3b\xf1j\x17\xfe\xe6\xe4\xd1\xf8\xe4\x97\xab\xf9E\x87U\x1e\xeb\tn\x17\xe0Ew\xe8\xab\xc0*W\x9d\x8e|^\xa3W\x83)\xbe\\\xd1\xbf#\xb0\xb2a\xdc\xda>\xc4\xa6\x87\xb7*\x97-\xbaT\xcf\xa6m\xbe.\xe6\xfb\x8b\x99\xf3\x96w\x02\xeb\x83C\xbb\xaa\x18\xfbM\xd3\xb4\xed\x16\xff\x17\xf31\n\x81\xf5\x9e\xb3\xc5\xabH\xcb\r\xbe\xf5\xa3I\xfe\xd9\xc5\xe3\xa4Jk5\xc1\xd5\xea\xd1\x17\x81\x955\xeb\x8d\xa4v\xa9\xefz+\xf5n\xbb\xeau/\xe2;\x02+\x8d\xc1\x93\xef&{\xdd\xbb\x9d\xe34o\'>[\x8d\xfb\x8b\x99\xf3\xeeX\xfce\x83?n\xfc\xe6\x00\xe7>q7\x1bv\x93\xf8b>F\x11<\xb0\xb2\xfe\xe0\xd5\xf05\xba\xf8\x80\xfa?\xdb\xff\xe6\xed\xae\xe4\xb2>Z\xcb\xf2z\xf5\xf6y\x80\xd9\xa2E\xbb\x03:_,^\xeb\x16\xe3>\xdf\xa3G\x93\xf2\xa0W\x92mKo*\x1f\xb4\x18\x9dTy<\xb6\x8b\xe8\xaa\x83\xf1E`\xa5m\'p\\\xa37\xdb9\xc5b\xab\xebpD\xbb!\xf7Om\'\xae\x9e\x06\xd1\xbf\x92M\xe9W\x0f\x91~<\xcd\xf9<Y\xe5\xb6\xfcY`\xf5C\\\xcd\xa5\x173g\x1e\x83\xe54\xae\xc6zQ\xeax,W;+\x8f\xde\xfa\xec\x16g\x87\x8d\xb7\xad\xbb\x11m\x97\xbci\xb1\xdd-\xd9/\xdb\xe4\x8b\xc0Z.\xfe\xaf\xe6c\xf7z\xdf\xf6q\xba\xb4\xff\x03B\x07V5\x1d"\xea\xbfG\x8f\x02kl\x93\xf6+\xfe|\xfcl\xfc\x1a\xce\xc7\x01\x0e\xad\xab\xe9\xe1\xf8\x11\xeeZdO\xd3\xe7\xbb~4\xca\xf6\x9d\x92\xed\x98/\x86\x9b\x9f\xb4X\xbc\xf9\xb0\xcav\x0e\xf4\x85\xe7\x9e\xdd\xeb\xc0\xaa\xf6\x9bz\xfd\x1a]mF \xaf\xaa\xf5\xc4lG\xb4\x1dry6q\xd5j\xb4\xda})\xd9\xb2\xd6\xc94/\xa6#\xd

In [9]:
all_dwi_visqc = dwi_w.get_changed_df()
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))

,bvals,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec,level_0
0,0,0.0,0.0,0.0,True,0.0,0.0,True,0.0,0.0,True,0.0,0.0,0.0,0
1,2000,1.0,1.0,0.0,True,1.0,0.0,True,0.0,0.0,True,1.0,0.0,0.0,1
2,2000,2.0,2.0,0.0,True,2.0,0.0,True,1.0,0.0,True,0.0,1.0,0.0,2
3,2000,3.0,3.0,0.0,True,3.0,0.0,True,2.0,0.0,True,0.0,0.0,1.0,3
4,2000,4.0,4.0,0.0,True,4.0,0.0,True,3.0,0.0,True,-0.625,-0.777,0.076,4
5,2000,5.0,5.0,0.0,True,5.0,0.0,True,4.0,0.0,True,-0.745,0.623,0.237,5
6,2000,6.0,6.0,0.0,True,6.0,0.0,True,5.0,0.0,True,0.025,-0.095,0.995,6
7,2000,7.0,7.0,0.0,True,7.0,NaN,True,6.0,1.0,True,-0.853,-0.004,-0.522,7
8,2000,NaN,8.0,NaN,True,NaN,NaN,True,7.0,0.0,True,-0.155,0.904,-0.399,8
9,2000,NaN,NaN,NaN,True,NaN,NaN,True,8.0,0.0,True,-0.025,-0.716,-0.698,9
